In [19]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import time
import pickle

In [20]:
import plotly
import plotly.graph_objs as go
import chart_studio.plotly as py
from chart_studio.plotly import plot, iplot
import plotly.express as px

In [21]:
# Setting pandas to display columns
pd.set_option('display.max_columns', None)

In [22]:
nfl_small2_end_of_drive = pd.read_csv('nfl_small_end_of_drive.csv',index_col=1)
nfl_small2_end_of_drive['KC'] = nfl_small2_end_of_drive['KC'].astype('str')
nfl_small2_end_of_drive['LAC'] = nfl_small2_end_of_drive['LAC'].astype('str')

In [23]:
# list(nfl_small2_end_of_drive.columns)

In [24]:
# Getting teams to add team matrix for easier filtering
teams = list(nfl_small2_end_of_drive.groupby('posteam').sum().index)
teams[:5]

['ARI', 'ATL', 'BAL', 'BUF', 'CAR']

In [25]:
nfl_small2_end_of_drive.head(5)

,index,game_id,game_date,time,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,qtr,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,drive,sp,down,goal_to_go,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,pass_length,air_yards,yards_after_catch,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,td_team,total_home_score,total_away_score,posteam_score,defteam_score,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,fumble,complete_pass,Start_Time,year,diff,Estimated_Time,Estimated Time (EST),Estimated_Hour,Air Pressure (hPa),City,Dewpoint (°C),Field,Humidity (%),Precipitation (mm),Roof,Team Abbreviation,Temperature (°C),Time (EST),Time (GMT),Unnamed: 0_y,Wind Direction (deg),Wind Speed (km/h),added_time,game_name,end_of_drive,end_of_game,INT,ARI,ATL,BAL,BUF,CAR,CHI,CIN,CLE,DAL,DEN,DET,GB,HOU,IND,JAX,KC,LA,LAC,MIA,MIN,NE,NO,NYG,NYJ,OAK,PHI,PIT,SEA,SF,TB,TEN,WAS,week
play_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
36,270409,2015091000,2015-09-10 00:00:00,15:00,900.0,1800.0,3600.0,Half1,0,1,NE,PIT,PIT,away,NE,NE,35.0,1,0,NaN,0.0,NE 35,0,0,S.Gostkowski kicks 65 yards from NE 35 to end ...,kickoff,0.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-10 20:40:00,2015.0,0 days 00:00:00.000000000,2015-09-10 20:40:00,2015-09-10 20:40:00,2015-09-10 21:00:00,1010.6,Foxborough,16.1,FieldTurf CORE,87.0,0.3,Open,NE,18.3,2015-09-10 21:00:00,2015-09-11 01:00:00,NaN,30.0,20.5,0,PIT @ NE 2015-09-10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,H,0,0,0,0,0,A,0,0,0,0,0,1
51,270410,2015091000,2015-09-10 00:00:00,15:00,900.0,1800.0,3600.0,Half1,0,1,NE,PIT,PIT,away,NE,PIT,80.0,1,0,1.0,0.0,PIT 20,10,18,(15:00) De.Williams right tackle to PIT 38 for...,run,18.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-10 20:40:00,2015.0,0 days 00:00:00.000000000,2015-09-10 20:40:00,2015-09-10 20:40:00,2015-09-10 21:00:00,1010.6,Foxborough,16.1,FieldTurf CORE,87.0,0.3,Open,NE,18.3,2015-09-10 21:00:00,2015-09-11 01:00:00,NaN,30.0,20.5,0,PIT @ NE 2015-09-10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,H,0,0,0,0,0,A,0,0,0,0,0,1
72,270411,2015091000,2015-09-10 00:00:00,14:21,861.0,1761.0,3561.0,Half1,0,1,NE,PIT,PIT,away,NE,PIT,62.0,1,0,1.0,0.0,PIT 38,10,31,(14:21) B.Roethlisberger pass short right to A...,pass,9.0,short,-4.0,13.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2015-09-10 20:40:00,2015.0,0 days 00:01:57.000000000,2015-09-10 20:41:57,2015-09-10 20:41:57,2015-09-10 21:00:00,1010.6,Foxborough,16.1,FieldTurf CORE,87.0,0.3,Open,NE,18.3,2015-09-10 21:00:00,2015-09-11 01:00:00,NaN,30.0,20.5,0,PIT @ NE 2015-09-10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,H,0,0,0,0,0,A,0,0,0,0,0,1
101,270412,2015091000,2015-09-10 00:00:00,14:04,844.0,1744.0,3544.0,Half1,0,1,NE,PIT,PIT,away,NE,PIT,53.0,1,0,2.0,0.0,PIT 47,1,31,(14:04) De.Williams right guard to NE 49 for 4...,run,4.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-10 20:40:00,2015.0,0 days 00:02:48.000000000,2015-09-10 20:42:48,2015-09-10 20:42:48,2015-09-10 21:00:00,1010.6,Foxborough,16.1,FieldTurf CORE,87.0,0.3,Open,NE,18.3,2015-09-10 21:00:00,2015-09-11 01:00:00,NaN,30.0,20.5,0,PIT @ NE 2015-09-10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,H,0,0,0,0,0,A,0,0,0,0,0,1
122,270413,2015091000,2015-09-10 00:00:00,13:26,806.0,1706.0,3506.0,Half1,0,1,NE,PIT,PIT,away,NE,NE,49.0,1,0,1.0,0.0,NE 49,10,45,(13:26) B.Roethlisberger pass short right to H...,pass,14.0,short,9.0,5.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2015-09-10 20:40:00,2015.0,0 days 00:04:42.000000000,2015-09-10 20:44:42,2015-09-10 20:44:42,2015-09-10 21:00:00,1010.6,Foxborough,16.1,FieldTurf CORE,87.0,0.3,Open,NE,18.3,2015-09-10 21:00:00,2015-0

In [26]:
# Checking this game, the yard totals look right, but the overall don't match online either 
# UPDATE: ydsnet is how far down the field the team drove, including penalties
# https://www.pro-football-reference.com/boxscores/201709100cle.htm#all_vis_drives
# nfl_small2_end_of_drive[(nfl_small2_end_of_drive['home_team']=='CLE')\
#                         & (nfl_small2_end_of_drive['week'] == 1)].to_csv('nfl_small_yard_explore.csv')

In [27]:
x_axis = nfl_small2_end_of_drive[(nfl_small2_end_of_drive['posteam']=='CLE')&\
                                 (nfl_small2_end_of_drive['year']==2017)].groupby('week').sum().index

y_axis = nfl_small2_end_of_drive[(nfl_small2_end_of_drive['posteam']=='CLE') &\
                        (nfl_small2_end_of_drive['end_of_drive']==1)&\
                        (nfl_small2_end_of_drive['year']==2017)]\
                        .groupby('week').sum()['ydsnet'].values

x_axis2 = nfl_small2_end_of_drive[(nfl_small2_end_of_drive['posteam']=='CLE')&\
                                 (nfl_small2_end_of_drive['year']==2018)].groupby('week').sum().index

y_axis2 = nfl_small2_end_of_drive[(nfl_small2_end_of_drive['posteam']=='CLE') &\
                        (nfl_small2_end_of_drive['end_of_drive']==1)&\
                        (nfl_small2_end_of_drive['year']==2018)]\
                        .groupby('week').sum()['ydsnet'].values

x_axis3 = nfl_small2_end_of_drive[(nfl_small2_end_of_drive['posteam']=='CLE')&\
                                 (nfl_small2_end_of_drive['year']==2016)].groupby('week').sum().index

y_axis3 = nfl_small2_end_of_drive[(nfl_small2_end_of_drive['posteam']=='CLE') &\
                        (nfl_small2_end_of_drive['end_of_drive']==1)&\
                        (nfl_small2_end_of_drive['year']==2016)]\
                        .groupby('week').sum()['ydsnet'].values

In [28]:
trace2016 = go.Scatter(
      x = x_axis3, 
      y = y_axis3, 
      mode = 'lines', 
      name = '2016',
      type = "scatter"
    )



trace2017 = go.Scatter(
      x = x_axis, 
      y = y_axis, 
      mode = 'lines', 
      name = '2017',
      type = "scatter"
    )

trace2018 = go.Scatter(
      x = x_axis2, 
      y = y_axis2, 
      mode = 'lines', 
      name = '2018',
      type = "scatter"
    )


layout = go.Layout(
    title = 'End of Game Yards: {}'.format('CLE'),
    xaxis= dict(title= 'Game',ticklen= 1,zeroline= False),
    yaxis= dict(title= 'Yards',ticklen= 5,zeroline= False)
)

fig = go.Figure(data = [trace2016,trace2017,trace2018], layout = layout)

fig.update_layout(
    title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig

In [29]:
nfl_small2_end_of_drive.head(3)

,index,game_id,game_date,time,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,qtr,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,drive,sp,down,goal_to_go,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,pass_length,air_yards,yards_after_catch,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,td_team,total_home_score,total_away_score,posteam_score,defteam_score,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,fumble,complete_pass,Start_Time,year,diff,Estimated_Time,Estimated Time (EST),Estimated_Hour,Air Pressure (hPa),City,Dewpoint (°C),Field,Humidity (%),Precipitation (mm),Roof,Team Abbreviation,Temperature (°C),Time (EST),Time (GMT),Unnamed: 0_y,Wind Direction (deg),Wind Speed (km/h),added_time,game_name,end_of_drive,end_of_game,INT,ARI,ATL,BAL,BUF,CAR,CHI,CIN,CLE,DAL,DEN,DET,GB,HOU,IND,JAX,KC,LA,LAC,MIA,MIN,NE,NO,NYG,NYJ,OAK,PHI,PIT,SEA,SF,TB,TEN,WAS,week
play_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
36,270409,2015091000,2015-09-10 00:00:00,15:00,900.0,1800.0,3600.0,Half1,0,1,NE,PIT,PIT,away,NE,NE,35.0,1,0,NaN,0.0,NE 35,0,0,S.Gostkowski kicks 65 yards from NE 35 to end ...,kickoff,0.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-10 20:40:00,2015.0,0 days 00:00:00.000000000,2015-09-10 20:40:00,2015-09-10 20:40:00,2015-09-10 21:00:00,1010.6,Foxborough,16.1,FieldTurf CORE,87.0,0.3,Open,NE,18.3,2015-09-10 21:00:00,2015-09-11 01:00:00,NaN,30.0,20.5,0,PIT @ NE 2015-09-10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,H,0,0,0,0,0,A,0,0,0,0,0,1
51,270410,2015091000,2015-09-10 00:00:00,15:00,900.0,1800.0,3600.0,Half1,0,1,NE,PIT,PIT,away,NE,PIT,80.0,1,0,1.0,0.0,PIT 20,10,18,(15:00) De.Williams right tackle to PIT 38 for...,run,18.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-10 20:40:00,2015.0,0 days 00:00:00.000000000,2015-09-10 20:40:00,2015-09-10 20:40:00,2015-09-10 21:00:00,1010.6,Foxborough,16.1,FieldTurf CORE,87.0,0.3,Open,NE,18.3,2015-09-10 21:00:00,2015-09-11 01:00:00,NaN,30.0,20.5,0,PIT @ NE 2015-09-10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,H,0,0,0,0,0,A,0,0,0,0,0,1
72,270411,2015091000,2015-09-10 00:00:00,14:21,861.0,1761.0,3561.0,Half1,0,1,NE,PIT,PIT,away,NE,PIT,62.0,1,0,1.0,0.0,PIT 38,10,31,(14:21) B.Roethlisberger pass short right to A...,pass,9.0,short,-4.0,13.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2015-09-10 20:40:00,2015.0,0 days 00:01:57.000000000,2015-09-10 20:41:57,2015-09-10 20:41:57,2015-09-10 21:00:00,1010.6,Foxborough,16.1,FieldTurf CORE,87.0,0.3,Open,NE,18.3,2015-09-10 21:00:00,2015-09-11 01:00:00,NaN,30.0,20.5,0,PIT @ NE 2015-09-10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,H,0,0,0,0,0,A,0,0,0,0,0,1


In [30]:
from plotly.subplots import make_subplots

In [31]:
# Includes Penalty Yards with current approach
fig = make_subplots(rows=6, cols=6,subplot_titles=teams)

teams = list(nfl_small2_end_of_drive.groupby('posteam').sum().index)
years = [2015,2016,2017,2018]
row = 1
col = 1

for team in teams:
    for year in years:
    
        x_axis = nfl_small2_end_of_drive[(nfl_small2_end_of_drive['posteam']==team) &\
                                    (nfl_small2_end_of_drive['year']==year)].groupby('week').sum().index
    
        y_axis = nfl_small2_end_of_drive[(nfl_small2_end_of_drive['posteam']==team) &\
#                         (nfl_small2_end_of_drive['end_of_drive']==1)&\
                        (nfl_small2_end_of_drive['year']==year)]\
                        .groupby('week').sum()['yards_gained'].values
        
        if year == 2015:
            color = 'black'
        elif year == 2016:
            color = 'red'
        elif year == 2017:
            color = 'green'
        else:
            color = 'blue'
    
        fig.add_trace(
        go.Scatter(
            x = x_axis, 
            y = y_axis, 
            mode = 'lines', 
            name = team,
            type = "scatter",line_color=color),
        row=row, col=col
            ).update_yaxes(range=[0, 1000], dtick=250, row=row, col=col)
    
    if col == 6:
        if row == 6:
            break
        else:
            row += 1
            col = 1
    else:
        col += 1
        
        
fig.update_layout(height=700, width=1000, template='plotly')\
    .update_xaxes(zeroline= False,showticklabels=False)\
    .update_yaxes(tickfont=dict(size=9))\
    .update_layout(showlegend=False,title={'text': "Team Yards Per Game",
                                        'y':0.95,
                                        'x':0.5,
                                        'xanchor': 'center',
                                        'yanchor': 'top'})

In [53]:
# nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
#                                             (nfl_small2_end_of_drive['year'] == 2017) &\
#                                             (nfl_small2_end_of_drive['PHI'] == 'H')]

In [54]:
# nfl_small2_end_of_drive[(nfl_small2_end_of_drive['game_id'] == 2017092409)].tail(20)

# Start of Report Team Stat Background

In [55]:
nfl_small2_end_of_drive['home_team_win'] = np.where(nfl_small2_end_of_drive['total_home_score']\
                              > nfl_small2_end_of_drive['total_away_score'],1,0)

nfl_small2_end_of_drive['away_team_win'] = np.where(nfl_small2_end_of_drive['total_home_score']\
                              < nfl_small2_end_of_drive['total_away_score'],1,0)

In [56]:
# Team wins overall for sorting
team_wins_overall = []

for team in teams:
    home_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                            (nfl_small2_end_of_drive[team] == 'H')
                                           ]['home_team_win'])

    away_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                                 (nfl_small2_end_of_drive[team] == 'A')
                                           ]['away_team_win'])
    
    all_wins = home_wins + away_wins
    
    team_wins_overall.append(round(all_wins,0))
    
team_by_wins = pd.DataFrame({'team':teams,'wins':team_wins_overall}).sort_values(by=['wins'])

In [57]:
bins = [0, 12, 19, 32]
labels = ['12 or Less', '19 or Less','20+ Wins']
team_by_wins['binned'] = pd.cut(team_by_wins['wins'], bins, labels=labels)

team_by_wins.sample(10)

,team,wins,binned
3,BUF,14,19 or Less
13,IND,12,12 or Less
10,DET,14,19 or Less
0,ARI,11,12 or Less
2,BAL,17,19 or Less
7,CLE,6,12 or Less
11,GB,12,12 or Less
16,LA,22,20+ Wins
19,MIN,21,20+ Wins
8,DAL,17,19 or Less


In [58]:
# Team wins by year
team_wins_2017 = []
for team in team_by_wins['team']:
    home_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                            (nfl_small2_end_of_drive['year'] == 2017) &\
                                            (nfl_small2_end_of_drive[team] == 'H')
                                           ]['home_team_win'])

    away_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                                 (nfl_small2_end_of_drive['year'] == 2017) &\
                                                 (nfl_small2_end_of_drive[team] == 'A')
                                           ]['away_team_win'])
    
    all_wins = home_wins + away_wins
    
    team_wins_2017.append(round(all_wins,0))
    
team_wins_2018 = []
for team in team_by_wins['team']:
    home_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                            (nfl_small2_end_of_drive['year'] == 2018) &\
                                            (nfl_small2_end_of_drive[team] == 'H')
                                           ]['home_team_win'])

    away_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                                 (nfl_small2_end_of_drive['year'] == 2018) &\
                                                 (nfl_small2_end_of_drive[team] == 'A')
                                           ]['away_team_win'])
    
    all_wins = home_wins + away_wins
    
    team_wins_2018.append(round(all_wins,0))

In [59]:
fig = go.Figure(data=[go.Bar(name='2017', x = team_by_wins['team'], y = team_wins_2017),\
                      go.Bar(name='2018', x = team_by_wins['team'], y = team_wins_2018)])

layout = go.Layout(
    title = 'Total Wins (2017 & 2018)',
    xaxis= dict(title= 'Team',ticklen= 1,zeroline= False),
    yaxis= dict(title= 'Wins',ticklen= 5,zeroline= False)
)

fig.update_layout(
    title={
        'text':'Total Wins by Year',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        barmode='group')
fig

You have a few distinct groups of teams based on number of wins. You have the "underacheivers" with 6 or less wins per season, you have the "middle of the pack" with 10 or less wins per season, and you have the "overachievers" with 10+ wins per season.

In [60]:
yd_avg2017 = []
for team in team_by_wins['team']:
    yd_avg2017.append(round(np.mean(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['posteam']==team) &\
                                                        (nfl_small2_end_of_drive['year']==2017)&\
                                                        (nfl_small2_end_of_drive['end_of_drive']==1)]\
                    .groupby('week').sum()['ydsnet'].values),0))
    
yd_avg2018 = []
for team in team_by_wins['team']:
    yd_avg2018.append(round(np.mean(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['posteam']==team) &\
                                                        (nfl_small2_end_of_drive['year']==2018)&\
                                                        (nfl_small2_end_of_drive['end_of_drive']==1)]\
                    .groupby('week').sum()['ydsnet'].values),0))

In [61]:
fig = go.Figure(data=[go.Bar(name='2017', x = team_by_wins['team'], y = yd_avg2017),\
                      go.Bar(name='2018', x = team_by_wins['team'], y = yd_avg2018)])

layout = go.Layout(
    title = 'Avg Yard Per Game',
    xaxis= dict(title= 'Game',ticklen= 1,zeroline= False),
    yaxis= dict(title= 'Yards',ticklen= 5,zeroline= False)
)

fig.update_layout(
    title={
        'text':'Avg Yards Per Game',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    barmode='group')
fig

These Win buckets do not quite correlate with a team's ability to move the ball

In [62]:
def_yd_avg2017 = []
for team in team_by_wins['team']:
    def_yd_avg2017.append(round(np.mean(nfl_small2_end_of_drive[(nfl_small2_end_of_drive[team] != '0') &\
                                                        (nfl_small2_end_of_drive['posteam']!=team) &\
                                                        (nfl_small2_end_of_drive['year']==2017)&\
                                                        (nfl_small2_end_of_drive['end_of_drive']==1)]\
                    .groupby('week').sum()['ydsnet'].values),0))
    
def_yd_avg2018 = []
for team in team_by_wins['team']:
    def_yd_avg2018.append(round(np.mean(nfl_small2_end_of_drive[(nfl_small2_end_of_drive[team] != '0') &\
                                                        (nfl_small2_end_of_drive['posteam']!=team) &\
                                                        (nfl_small2_end_of_drive['year']==2018)&\
                                                        (nfl_small2_end_of_drive['end_of_drive']==1)]\
                    .groupby('week').sum()['ydsnet'].values),0))

In [63]:
fig = go.Figure(data=[go.Bar(name='2017', x = team_by_wins['team'], y = def_yd_avg2017),\
                      go.Bar(name='2018', x = team_by_wins['team'], y = def_yd_avg2018)])

layout = go.Layout(
    title = 'Avg Yard Per Game',
    xaxis= dict(title= 'Game',ticklen= 1,zeroline= False),
    yaxis= dict(title= 'Yards',ticklen= 5,zeroline= False)
)

fig.update_layout(
    title={
        'text':'Avg Yards Allowed Per Game By Team',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    barmode='group')
fig

Teams who tend to perform more poorly seem to give up more yards per game.

In [ ]:
# nfl_small2_end_of_drive[(nfl_small2_end_of_drive['posteam']=='CLE') &\
#                                                     (nfl_small2_end_of_drive['year']==2017)&\
#                                                     (nfl_small2_end_of_drive['end_of_drive']==1)]\
#                     .groupby('week').sum()['ydsnet'].values

In [67]:
avg_yard_diff_2017 = []
for team in team_by_wins['team']:
    
    team_yd_2017 = nfl_small2_end_of_drive[(nfl_small2_end_of_drive['posteam']==team) &\
                                                    (nfl_small2_end_of_drive['year']==2017)&\
                                                    (nfl_small2_end_of_drive['end_of_drive']==1)]\
                    .groupby('week').sum()['ydsnet'].values
    
    
    team_def_yd_2017 = nfl_small2_end_of_drive[(nfl_small2_end_of_drive[team] != '0') &\
                                                        (nfl_small2_end_of_drive['posteam']!=team) &\
                                                        (nfl_small2_end_of_drive['year']==2017)&\
                                                        (nfl_small2_end_of_drive['end_of_drive']==1)]\
                    .groupby('week').sum()['ydsnet'].values
    
    team_yard_diff_2017 = [a_i - b_i for a_i, b_i in zip(team_yd_2017, team_def_yd_2017)]
    
    
    avg_yard_diff_2017.append(np.round(np.mean(team_yard_diff_2017),2))
    
    
avg_yard_diff_2018 = []
for team in team_by_wins['team']:
    
    team_yd_2018 = nfl_small2_end_of_drive[(nfl_small2_end_of_drive['posteam']==team) &\
                                                    (nfl_small2_end_of_drive['year']==2018)&\
                                                    (nfl_small2_end_of_drive['end_of_drive']==1)]\
                    .groupby('week').sum()['ydsnet'].values
    
    
    team_def_yd_2018 = nfl_small2_end_of_drive[(nfl_small2_end_of_drive[team] != '0') &\
                                                        (nfl_small2_end_of_drive['posteam']!=team) &\
                                                        (nfl_small2_end_of_drive['year']==2018)&\
                                                        (nfl_small2_end_of_drive['end_of_drive']==1)]\
                    .groupby('week').sum()['ydsnet'].values
    
    team_yard_diff_2018 = [a_i - b_i for a_i, b_i in zip(team_yd_2018, team_def_yd_2018)]
    
    
    avg_yard_diff_2018.append(np.round(np.mean(team_yard_diff_2018),2))
    
    

In [68]:
fig = go.Figure(data=[go.Bar(name='2017', x = team_by_wins['team'], y = avg_yard_diff_2017),\
                      go.Bar(name='2018', x = team_by_wins['team'], y = avg_yard_diff_2018)])

layout = go.Layout(
    title = 'Avg Yard Per Game',
    xaxis= dict(title= 'Game',ticklen= 1,zeroline= False),
    yaxis= dict(title= 'Yards',ticklen= 5,zeroline= False)
)

fig.update_layout(
    title={
        'text':'Yard Differential Per Game By Team',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    barmode='group')
fig

And as expected, average yard differential per game is directly correlated to a team's chance of winning.

Looking at Wins and Yard Differential together, good and bad teams are impervious to the random aspects of the game. Teams closer to average may be more affected by changes to weather or other factors...

In [69]:
pt_avg2017 = []
for team in team_by_wins['team']:
    
    home_list = list(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                        (nfl_small2_end_of_drive[team] == 'H')&(nfl_small2_end_of_drive['year'] == 2017)
                       ]['total_home_score'])

    away_list = list(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                        (nfl_small2_end_of_drive[team] == 'A')&(nfl_small2_end_of_drive['year'] == 2017)
                       ]['total_away_score'])
    
    new_list = home_list + away_list
    
    pt_avg2017.append(round(np.mean(new_list),0))

pt_avg2018 = []

for team in team_by_wins['team']:
    
    home_list = list(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                        (nfl_small2_end_of_drive[team] == 'H')&(nfl_small2_end_of_drive['year'] == 2018)
                       ]['total_home_score'])

    away_list = list(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                        (nfl_small2_end_of_drive[team] == 'A')&(nfl_small2_end_of_drive['year'] == 2018)
                       ]['total_away_score'])
    
    new_list = home_list + away_list
    
    pt_avg2018.append(round(np.mean(new_list),0))
    

In [70]:
fig = go.Figure(data=[go.Bar(name='2017', x = team_by_wins['team'], y = pt_avg2017),\
                      go.Bar(name='2018', x = team_by_wins['team'], y = pt_avg2018)])

layout = go.Layout(
    title = 'Avg Points Per Game',
    xaxis= dict(title= 'Game',ticklen= 1,zeroline= False),
    yaxis= dict(title= 'Points',ticklen= 5,zeroline= False)
)

fig.update_layout(
    title={
        'text':'Avg Points Per Game By Team',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        barmode='group')
fig

Average points per game do not directly correlate to wins, and yard differential appears to be a better indicator. Yard differential may even help us understand weather impacts.

In [71]:
team_pass_yards_2017 = []
team_run_yards_2017 = []
for team in team_by_wins['team']:
    pass_yards_2017 = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['posteam']==team) &\
                                             (nfl_small2_end_of_drive['year'] == 2017) &\
                                            (nfl_small2_end_of_drive['play_type'] == 'pass')]['yards_gained'])/16

    run_yards_2017 = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['posteam']==team) &\
                                            (nfl_small2_end_of_drive['year'] == 2017) &\
                                            (nfl_small2_end_of_drive['play_type'] == 'run')]['yards_gained'])/16

    
    
    team_pass_yards_2017.append(pass_yards_2017)
    team_run_yards_2017.append(run_yards_2017)


team_pass_yards_2018 = []
team_run_yards_2018 = []
for team in team_by_wins['team']:
    pass_yards_2018 = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['posteam']==team) &\
                                             (nfl_small2_end_of_drive['year'] == 2018) &\
                                            (nfl_small2_end_of_drive['play_type'] == 'pass')]['yards_gained'])/14

    run_yards_2018 = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['posteam']==team) &\
                                            (nfl_small2_end_of_drive['year'] == 2018) &\
                                            (nfl_small2_end_of_drive['play_type'] == 'run')]['yards_gained'])/14

    
    
    team_pass_yards_2018.append(pass_yards_2018)
    team_run_yards_2018.append(run_yards_2018)

In [72]:
fig = go.Figure()


fig.add_trace(go.Scatter(name='2017',x=team_pass_yards_2017, y=team_run_yards_2017, mode='markers',\
                                   text=teams, marker=dict(size=12))
             )
              
fig.add_trace(go.Scatter(name='2018',x=team_pass_yards_2018, y=team_run_yards_2018, mode='markers',\
                                   text=teams, marker=dict(size=12))
             )              
              

                               

fig.update_layout(
    title={
        'text':'Run to Pass Ratio (Avg Game Yards)',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis= dict(title= 'Passing Yards',ticklen= 1),
    yaxis= dict(title= 'Rushing Yards',ticklen= 5))

fig

In [73]:
pass_run_ratio_2017 = [round(x/y,2) for x,y in zip(team_pass_yards_2017, team_run_yards_2017)]
pass_run_ratio_2018 = [round(x/y,2) for x,y in zip(team_pass_yards_2018, team_run_yards_2018)]

In [74]:
fig = go.Figure(data = [go.Bar(name = '2017', x = team_by_wins['team'], y = pass_run_ratio_2017),\
                        go.Bar(name = '2018', x = team_by_wins['team'], y = pass_run_ratio_2018)
                       ])

layout = go.Layout(
    title = 'Pass to Run Ratio (2017 & 2018)',
    xaxis= dict(title= 'Team',ticklen= 1,zeroline= False),
    yaxis= dict(title= 'Pass/Run Ratio',ticklen= 5,zeroline= False)
)

fig.update_layout(
    title={
        'text':'Pass to Run Ratio (2017 & 2018)',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        barmode='group')

fig

In [75]:
nfl_small2_end_of_drive[nfl_small2_end_of_drive['INT'] == 1].head(3)

,index,game_id,game_date,time,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,qtr,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,drive,sp,down,goal_to_go,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,pass_length,air_yards,yards_after_catch,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,td_team,total_home_score,total_away_score,posteam_score,defteam_score,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,fumble,complete_pass,Start_Time,diff,Estimated_Time,Estimated Time (EST),Estimated_Hour,Air Pressure (hPa),City,Dewpoint (°C),Field,Humidity (%),Precipitation (mm),Roof,Team Abbreviation,Temperature (°C),Time (EST),Time (GMT),Wind Direction (deg),Wind Speed (km/h),added_time,game_name,end_of_drive,end_of_game,INT,ARI,ATL,BAL,BUF,CAR,CHI,CIN,CLE,DAL,DEN,DET,GB,HOU,IND,JAX,KC,LA,LAC,MIA,MIN,NE,NO,NYG,NYJ,OAK,PHI,PIT,SEA,SF,TB,TEN,WAS,year,week,home_team_win,away_team_win
play_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3101,362416,2017091000,2017-09-10 00:00:00,02:00,120.0,1020.0,1020.0,Half2,0,3,BUF,NYJ,NYJ,away,BUF,BUF,2.0,15,0,NaN,0.0,BUF 2,0,75,TWO-POINT CONVERSION ATTEMPT. J.McCown pass to...,pass,0.0,0,0.0,0.0,NaN,NaN,NaN,failure,NaN,14,12,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017-09-10 13:00:00,0 days 02:09:00.000000000,2017-09-10 15:09:00,2017-09-10 15:09:00,2017-09-10 15:00:00,1031.5,Orchard Park,6.7,A-Turf Titan 50,45.0,0.0,Open,BUF,18.9,2017-09-10 15:00:00,2017-09-10 19:00:00,30.0,14.8,0,NYJ @ BUF 2017-09-10,1,0,1,0,0,0,H,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,A,0,0,0,0,0,0,0,0,2017,1,1,0
3765,362443,2017091000,2017-09-10 00:00:00,06:44,404.0,404.0,404.0,Half2,0,4,BUF,NYJ,NYJ,away,BUF,BUF,37.0,19,0,3.0,0.0,BUF 37,8,2,(6:44) (Shotgun) J.McCown pass short right int...,pass,0.0,short,14.0,0.0,NaN,NaN,NaN,NaN,NaN,21,12,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-09-10 13:00:00,0 days 02:39:48.000000000,2017-09-10 15:39:48,2017-09-10 15:39:48,2017-09-10 16:00:00,1031.1,Orchard Park,6.7,A-Turf Titan 50,45.0,0.0,Open,BUF,18.9,2017-09-10 16:00:00,2017-09-10 20:00:00,10.0,11.2,0,NYJ @ BUF 2017-09-10,1,0,1,0,0,0,H,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,A,0,0,0,0,0,0,0,0,2017,1,1,0
4344,362465,2017091000,2017-09-10 00:00:00,01:44,104.0,104.0,104.0,Half2,0,4,BUF,NYJ,NYJ,away,BUF,NYJ,59.0,23,0,1.0,0.0,NYJ 41,10,0,(1:44) (Shotgun) J.McCown pass deep left inten...,pass,0.0,deep,18.0,0.0,NaN,NaN,NaN,NaN,NaN,21,12,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-09-10 13:00:00,0 days 02:54:48.000000000,2017-09-10 15:54:48,2017-09-10 15:54:48,2017-09-10 16:00:00,1031.1,Orchard Park,6.7,A-Turf Titan 50,45.0,0.0,Open,BUF,18.9,2017-09-10 16:00:00,2017-09-10 20:00:00,10.0,11.2,0,NYJ @ BUF 2017-09-10,1,0,1,0,0,0,H,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,A,0,0,0,0,0,0,0,0,2017,1,1,0


In [76]:
len(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['posteam']=='CLE') &\
                                             (nfl_small2_end_of_drive['year'] == 2017) &\
                                            ((nfl_small2_end_of_drive['INT'] == 1) |\
                                             (nfl_small2_end_of_drive['fumble'] == 1))
                           ].index)

52

In [77]:
len(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['posteam']=='CLE') &\
                                             (nfl_small2_end_of_drive['year'] == 2017)
                           ].index)

1367

In [78]:
TO_rate_2017 = []
for team in team_by_wins['team']:
    int_2017 = len(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['posteam']==team) &\
                                             (nfl_small2_end_of_drive['year'] == 2017) &\
                                            ((nfl_small2_end_of_drive['INT'] == 1) |\
                                             (nfl_small2_end_of_drive['fumble'] == 1))].index)

    off_play_2017 = len(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['posteam']==team) &\
                                             (nfl_small2_end_of_drive['year'] == 2017)
                           ].index)

    TO_rate_2017.append(round(int_2017/off_play_2017,3))
    
TO_rate_2018 = []
for team in team_by_wins['team']:
    int_2018 = len(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['posteam']==team) &\
                                             (nfl_small2_end_of_drive['year'] == 2018) &\
                                            ((nfl_small2_end_of_drive['INT'] == 1) |\
                                             (nfl_small2_end_of_drive['fumble'] == 1))].index)

    off_play_2018 = len(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['posteam']==team) &\
                                             (nfl_small2_end_of_drive['year'] == 2018)
                           ].index)

    TO_rate_2018.append(round(int_2018/off_play_2018,3))


In [79]:
TO_rate_2017[:4]

[0.038, 0.035, 0.039, 0.032]

In [80]:
fig = go.Figure(data = [go.Bar(name = '2017', x = team_by_wins['team'], y = TO_rate_2017),\
                        go.Bar(name = '2018', x = team_by_wins['team'], y = TO_rate_2018)
                       ])

layout = go.Layout(
    title = 'Turnover Rate per Year (2017 & 2018)',
    xaxis= dict(title= 'Team',ticklen= 1,zeroline= False),
    yaxis= dict(title= 'TO Rate',ticklen= 5,zeroline= False)
)

fig.update_layout(
    title={
        'text':'Turnover Rate per Year (2017 & 2018)',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        barmode='group')

fig

Also, as expected, teams more likely to win seem to turn the ball over less

In [81]:
df = nfl_small2_end_of_drive.groupby(['game_id','game_name']).count().reset_index()[['game_id','game_name']]

<h2>Feature Analysis for Weather</h2>

In [82]:
# Using a form of Joey's agg function
def my_aggfunc(x):
    values = {
        'pass_home':x[x['posteam_type']=='home']['play_type'].eq('pass').sum(),
        'pass_away':x[x['posteam_type']=='away']['play_type'].eq('pass').sum(),
        
        'run_home':x[x['posteam_type']=='home']['play_type'].eq('run').sum(),
        'run_away':x[x['posteam_type']=='away']['play_type'].eq('run').sum(),
        
        'TO_home':x[x['posteam_type']=='home']['INT'].sum() +\
        x[x['posteam_type']=='home']['fumble'].sum(),
        'TO_away':x[x['posteam_type']=='away']['INT'].sum() +\
        x[x['posteam_type']=='away']['fumble'].sum(),
        
        'yards_home':x[(x['posteam_type']=='home') & (x['end_of_drive']==1)]['ydsnet'].sum(),
        'yards_away':x[(x['posteam_type']=='away') & (x['end_of_drive']==1)]['ydsnet'].sum(),
        
        'fg_at_home':x[(x['posteam_type']=='home')&(x['play_type']=='field_goal')]['play_type'].count(),
        'fg_at_away':x[(x['posteam_type']=='away')&(x['play_type']=='field_goal')]['play_type'].count(),
        
        'fg_md_home':x[(x['posteam_type']=='home')&(x['play_type']=='field_goal')]['play_type'].count(),
        'fg_md_away':x[(x['posteam_type']=='away')&(x['play_type']=='field_goal')]['play_type'].count(),
        
        'pass_yards_home':x[(x['posteam_type']=='home')&(x['play_type']=='pass')]['yards_gained'].sum(),
        'pass_yards_away':x[(x['posteam_type']=='away')&(x['play_type']=='pass')]['yards_gained'].sum(),
        
        'run_yards_home':x[(x['posteam_type']=='home')&(x['play_type']=='run')]['yards_gained'].sum(),
        'run_yards_away':x[(x['posteam_type']=='away')&(x['play_type']=='run')]['yards_gained'].sum()
        
    }
    return pd.Series(values)

In [83]:
new_df = pd.DataFrame()
new_df = nfl_small2_end_of_drive.groupby(['game_id']).apply(my_aggfunc)


In [84]:
new_df['home_team'] = nfl_small2_end_of_drive.groupby('game_id')['home_team'].first()
new_df['away_team'] = nfl_small2_end_of_drive.groupby('game_id')['away_team'].first()
new_df['home_score'] = nfl_small2_end_of_drive.groupby('game_id')['total_home_score'].last()
new_df['away_score'] = nfl_small2_end_of_drive.groupby('game_id')['total_away_score'].last()

In [85]:
new_df.reset_index().head()

,game_id,pass_home,pass_away,run_home,run_away,TO_home,TO_away,yards_home,yards_away,fg_at_home,fg_at_away,fg_md_home,fg_md_away,pass_yards_home,pass_yards_away,run_yards_home,run_yards_away,home_team,away_team,home_score,away_score
0,2017090700,39.0,38.0,34.0,24.0,2.0,1.0,425.0,538.0,2.0,0.0,2.0,0.0,247.0,352.0,125.0,191.0,NE,KC,27,42
1,2017091000,29.0,41.0,39.0,14.0,2.0,3.0,400.0,199.0,1.0,2.0,1.0,2.0,218.0,176.0,193.0,39.0,BUF,NYJ,21,12
2,2017091001,44.0,32.0,19.0,21.0,3.0,0.0,306.0,372.0,1.0,3.0,1.0,3.0,176.0,308.0,125.0,66.0,CHI,ATL,17,23
3,2017091002,36.0,18.0,22.0,42.0,7.0,1.0,217.0,255.0,0.0,2.0,0.0,2.0,144.0,111.0,77.0,157.0,CIN,BAL,0,20
4,2017091003,37.0,37.0,26.0,14.0,2.0,3.0,293.0,287.0,1.0,0.0,1.0,0.0,180.0,255.0,59.0,43.0,CLE,PIT,18,20


In [86]:
# new_df['pass_ratio'] = new_df['']

In [87]:
away_cols = ['pass_away','run_away','TO_away','yards_away','away_team','home_score','away_score']

In [88]:
test_df = new_df.reset_index()[['game_id','home_team','away_team']]\
                .melt(id_vars='game_id',var_name = 'home_away',value_name="Team")


In [89]:
test_df[test_df['game_id']==2017090700]

,game_id,home_away,Team
0,2017090700,home_team,NE
480,2017090700,away_team,KC


In [90]:
test_df_home = test_df.merge(new_df.reset_index(),how='inner',left_on=['game_id','Team'],\
                             right_on=['game_id','home_team'])
# test_df = test_df.merge(new_df.reset_index(),how='left',left_on=['game_id','Team'],right_on=['game_id','away_team'])

In [91]:
# Setting variables for Home Team
test_df_home['team_against'] = test_df_home[test_df_home['home_away']=='home_team']['away_team']
test_df_home['score'] = test_df_home[test_df_home['home_away']=='home_team']['home_score']
test_df_home['score_against'] = test_df_home[test_df_home['home_away']=='home_team']['away_score']

test_df_home['yards'] = test_df_home[test_df_home['home_away']=='home_team']['yards_home']
test_df_home['yards_against'] = test_df_home[test_df_home['home_away']=='home_team']['yards_away']

test_df_home['run_plays'] = test_df_home[test_df_home['home_away']=='home_team']['run_home']
test_df_home['run_plays_against'] = test_df_home[test_df_home['home_away']=='home_team']['run_away']

test_df_home['pass_plays'] = test_df_home[test_df_home['home_away']=='home_team']['pass_home']
test_df_home['pass_plays_against'] = test_df_home[test_df_home['home_away']=='home_team']['pass_away']

test_df_home['to_for'] = test_df_home[test_df_home['home_away']=='home_team']['TO_home']
test_df_home['to_against'] = test_df_home[test_df_home['home_away']=='home_team']['TO_away']

test_df_home['fg_at'] = test_df_home[test_df_home['home_away']=='home_team']['fg_at_home']
test_df_home['fg_md'] = test_df_home[test_df_home['home_away']=='home_team']['fg_md_home']

test_df_home['fg_at_against'] = test_df_home[test_df_home['home_away']=='home_team']['fg_at_away']
test_df_home['fg_md_against'] = test_df_home[test_df_home['home_away']=='home_team']['fg_md_away']

test_df_home['pass_yards'] = test_df_home[test_df_home['home_away']=='home_team']['pass_yards_home']
test_df_home['pass_yards_against'] = test_df_home[test_df_home['home_away']=='home_team']['pass_yards_away']

test_df_home['run_yards'] = test_df_home[test_df_home['home_away']=='home_team']['run_yards_home']
test_df_home['run_yards_against'] = test_df_home[test_df_home['home_away']=='home_team']['run_yards_away']

# Add other field goal columns



In [92]:
test_df_home[test_df_home['game_id']==2017090700]

,game_id,home_away,Team,pass_home,pass_away,run_home,run_away,TO_home,TO_away,yards_home,yards_away,fg_at_home,fg_at_away,fg_md_home,fg_md_away,pass_yards_home,pass_yards_away,run_yards_home,run_yards_away,home_team,away_team,home_score,away_score,team_against,score,score_against,yards,yards_against,run_plays,run_plays_against,pass_plays,pass_plays_against,to_for,to_against,fg_at,fg_md,fg_at_against,fg_md_against,pass_yards,pass_yards_against,run_yards,run_yards_against
0,2017090700,home_team,NE,39.0,38.0,34.0,24.0,2.0,1.0,425.0,538.0,2.0,0.0,2.0,0.0,247.0,352.0,125.0,191.0,NE,KC,27,42,KC,27,42,425.0,538.0,34.0,24.0,39.0,38.0,2.0,1.0,2.0,2.0,0.0,0.0,247.0,352.0,125.0,191.0


In [93]:
# Creating away only df to merge records from hoem and away team stats
test_df_away = test_df.merge(new_df.reset_index(),how='inner',left_on=['game_id','Team'],right_on=['game_id','away_team'])
test_df_away[test_df_away['game_id']==2017090700]


,game_id,home_away,Team,pass_home,pass_away,run_home,run_away,TO_home,TO_away,yards_home,yards_away,fg_at_home,fg_at_away,fg_md_home,fg_md_away,pass_yards_home,pass_yards_away,run_yards_home,run_yards_away,home_team,away_team,home_score,away_score
0,2017090700,away_team,KC,39.0,38.0,34.0,24.0,2.0,1.0,425.0,538.0,2.0,0.0,2.0,0.0,247.0,352.0,125.0,191.0,NE,KC,27,42


In [94]:
# Setting variables for Away Team
test_df_away['team_against'] = test_df_away[test_df_away['home_away']=='away_team']['home_team']
test_df_away['score'] = test_df_away[test_df_away['home_away']=='away_team']['away_score']
test_df_away['score_against'] = test_df_away[test_df_away['home_away']=='away_team']['home_score']

test_df_away['yards'] = test_df_away[test_df_away['home_away']=='away_team']['yards_away']
test_df_away['yards_against'] = test_df_away[test_df_away['home_away']=='away_team']['yards_home']

test_df_away['run_plays'] = test_df_away[test_df_away['home_away']=='away_team']['run_away']
test_df_away['run_plays_against'] = test_df_away[test_df_away['home_away']=='away_team']['run_home']

test_df_away['pass_plays'] = test_df_away[test_df_away['home_away']=='away_team']['pass_away']
test_df_away['pass_plays_against'] = test_df_away[test_df_away['home_away']=='away_team']['pass_home']

test_df_away['to_for'] = test_df_away[test_df_away['home_away']=='away_team']['TO_away']
test_df_away['to_against'] = test_df_away[test_df_away['home_away']=='away_team']['TO_home']

test_df_away['fg_at'] = test_df_away[test_df_away['home_away']=='away_team']['fg_at_away']
test_df_away['fg_md'] = test_df_away[test_df_away['home_away']=='away_team']['fg_md_away']

test_df_away['fg_at_against'] = test_df_away[test_df_away['home_away']=='away_team']['fg_at_home']
test_df_away['fg_md_against'] = test_df_away[test_df_away['home_away']=='away_team']['fg_md_home']

test_df_away['pass_yards'] = test_df_away[test_df_away['home_away']=='away_team']['pass_yards_away']
test_df_away['pass_yards_against'] = test_df_away[test_df_away['home_away']=='away_team']['pass_yards_home']

test_df_away['run_yards'] = test_df_away[test_df_away['home_away']=='away_team']['run_yards_away']
test_df_away['run_yards_against'] = test_df_away[test_df_away['home_away']=='away_team']['run_yards_home']

test_df_away[test_df_away['game_id']==2017090700]

,game_id,home_away,Team,pass_home,pass_away,run_home,run_away,TO_home,TO_away,yards_home,yards_away,fg_at_home,fg_at_away,fg_md_home,fg_md_away,pass_yards_home,pass_yards_away,run_yards_home,run_yards_away,home_team,away_team,home_score,away_score,team_against,score,score_against,yards,yards_against,run_plays,run_plays_against,pass_plays,pass_plays_against,to_for,to_against,fg_at,fg_md,fg_at_against,fg_md_against,pass_yards,pass_yards_against,run_yards,run_yards_against
0,2017090700,away_team,KC,39.0,38.0,34.0,24.0,2.0,1.0,425.0,538.0,2.0,0.0,2.0,0.0,247.0,352.0,125.0,191.0,NE,KC,27,42,NE,42,27,538.0,425.0,24.0,34.0,38.0,39.0,1.0,2.0,0.0,0.0,2.0,2.0,352.0,247.0,191.0,125.0


In [95]:
# Creating cols filter for new df
new_cols = ['game_id','Team','team_against','score','score_against','yards','yards_against','run_plays',\
            'run_plays_against','pass_plays','pass_plays_against','to_for','to_against','fg_at',\
           'fg_md','fg_at_against','fg_md_against','pass_yards','pass_yards_against','run_yards','run_yards_against']


In [96]:
# Combining home and away dfs
combined_df = test_df_home[new_cols].append(test_df_away[new_cols],ignore_index=True)
combined_df[combined_df['game_id']==2017090700]

,game_id,Team,team_against,score,score_against,yards,yards_against,run_plays,run_plays_against,pass_plays,pass_plays_against,to_for,to_against,fg_at,fg_md,fg_at_against,fg_md_against,pass_yards,pass_yards_against,run_yards,run_yards_against
0,2017090700,NE,KC,27,42,425.0,538.0,34.0,24.0,39.0,38.0,2.0,1.0,2.0,2.0,0.0,0.0,247.0,352.0,125.0,191.0
480,2017090700,KC,NE,42,27,538.0,425.0,24.0,34.0,38.0,39.0,1.0,2.0,0.0,0.0,2.0,2.0,352.0,247.0,191.0,125.0


In [97]:
# Creating my target variable wins
combined_df.loc[combined_df['score'] > combined_df['score_against'],'win'] = 1

combined_df['win'].fillna(0,inplace=True)

In [98]:
combined_df[combined_df['game_id']==2017090700]

,game_id,Team,team_against,score,score_against,yards,yards_against,run_plays,run_plays_against,pass_plays,pass_plays_against,to_for,to_against,fg_at,fg_md,fg_at_against,fg_md_against,pass_yards,pass_yards_against,run_yards,run_yards_against,win
0,2017090700,NE,KC,27,42,425.0,538.0,34.0,24.0,39.0,38.0,2.0,1.0,2.0,2.0,0.0,0.0,247.0,352.0,125.0,191.0,0.0
480,2017090700,KC,NE,42,27,538.0,425.0,24.0,34.0,38.0,39.0,1.0,2.0,0.0,0.0,2.0,2.0,352.0,247.0,191.0,125.0,1.0


In [99]:
# Creating additional variables

combined_df['pass_run_ratio'] = round(combined_df['pass_plays'] / combined_df['run_plays'],2)
combined_df['yard_diff'] = combined_df['yards'] - combined_df['yards_against']
combined_df['to_diff'] = combined_df['to_for'] - combined_df['to_against']

combined_df['fg_rate'] = round(combined_df['fg_md'] / combined_df['fg_at'],2)
combined_df['fg_rate_against'] = round(combined_df['fg_md_against'] / combined_df['fg_at_against'],2)

# Setting fg rates to 50% for teams not attempting field goals
combined_df['fg_rate'].fillna(0.5,inplace=True)
combined_df['fg_rate_against'].fillna(0.5,inplace=True)

combined_df['pass_yds_per_at'] = combined_df['pass_yards'] / combined_df['pass_plays']
combined_df['run_yds_per_at'] = combined_df['run_yards'] / combined_df['run_plays']

combined_df['pass_yds_per_at_against'] = combined_df['pass_yards_against'] / combined_df['pass_plays_against']
combined_df['run_yds_per_at_against'] = combined_df['run_yards_against'] / combined_df['run_plays_against']


In [100]:
# Creating team encoding based off 2017/2018 wins
team_encode = {'CLE':1,'NYG':1,'OAK':1,'NYJ':1,'TB':1,'SF':1,'ARI':1,\
               'DEN':2,'CIN':2,'GB':2,'IND':2,'MIA':2,'JAX':3,'WAS':3,\
               'DET':3,'BUF':3,'HOU':3,'ATL':3,'CHI':3,'CAR':3,\
               'SEA':4,'BAL':4,'TEN':4,'DAL':4,'LAC':4,'PHI':4,'MIN':4,'PIT':5,\
               'KC':5,'NE':5,'LA':5,'NO':5}

In [101]:
combined_df['team_encode'] = combined_df.replace({'Team':team_encode})['Team']
combined_df['team_against_encode'] = combined_df.replace({'team_against':team_encode})['team_against']


In [102]:
combined_df[combined_df['game_id']==2017090700]

,game_id,Team,team_against,score,score_against,yards,yards_against,run_plays,run_plays_against,pass_plays,pass_plays_against,to_for,to_against,fg_at,fg_md,fg_at_against,fg_md_against,pass_yards,pass_yards_against,run_yards,run_yards_against,win,pass_run_ratio,yard_diff,to_diff,fg_rate,fg_rate_against,pass_yds_per_at,run_yds_per_at,pass_yds_per_at_against,run_yds_per_at_against,team_encode,team_against_encode
0,2017090700,NE,KC,27,42,425.0,538.0,34.0,24.0,39.0,38.0,2.0,1.0,2.0,2.0,0.0,0.0,247.0,352.0,125.0,191.0,0.0,1.15,-113.0,1.0,1.0,0.5,6.333333,3.676471,9.263158,7.958333,5,5
480,2017090700,KC,NE,42,27,538.0,425.0,24.0,34.0,38.0,39.0,1.0,2.0,0.0,0.0,2.0,2.0,352.0,247.0,191.0,125.0,1.0,1.58,113.0,-1.0,0.5,1.0,9.263158,7.958333,6.333333,3.676471,5,5


In [103]:
combined_df.columns

Index(['game_id', 'Team', 'team_against', 'score', 'score_against', 'yards',
       'yards_against', 'run_plays', 'run_plays_against', 'pass_plays',
       'pass_plays_against', 'to_for', 'to_against', 'fg_at', 'fg_md',
       'fg_at_against', 'fg_md_against', 'pass_yards', 'pass_yards_against',
       'run_yards', 'run_yards_against', 'win', 'pass_run_ratio', 'yard_diff',
       'to_diff', 'fg_rate', 'fg_rate_against', 'pass_yds_per_at',
       'run_yds_per_at', 'pass_yds_per_at_against', 'run_yds_per_at_against',
       'team_encode', 'team_against_encode'],
      dtype='object')

In [104]:
from sklearn.linear_model import LinearRegression

In [105]:
# Dependent variables
dep_var = ['score', 'score_against', 'yards',
       'yards_against', 'run_plays', 'run_plays_against', 'pass_plays',
       'pass_plays_against', 'to_for', 'to_against', 'fg_at', 'fg_md',
       'fg_at_against', 'fg_md_against', 'pass_yards', 'pass_yards_against',
       'run_yards', 'run_yards_against', 'pass_run_ratio', 'yard_diff',
       'to_diff', 'fg_rate', 'fg_rate_against', 'pass_yds_per_at',
       'run_yds_per_at', 'pass_yds_per_at_against', 'run_yds_per_at_against',
       'team_encode', 'team_against_encode']
X = pd.DataFrame(combined_df, columns = dep_var)
X.head(2)

,score,score_against,yards,yards_against,run_plays,run_plays_against,pass_plays,pass_plays_against,to_for,to_against,fg_at,fg_md,fg_at_against,fg_md_against,pass_yards,pass_yards_against,run_yards,run_yards_against,pass_run_ratio,yard_diff,to_diff,fg_rate,fg_rate_against,pass_yds_per_at,run_yds_per_at,pass_yds_per_at_against,run_yds_per_at_against,team_encode,team_against_encode
0,27,42,425.0,538.0,34.0,24.0,39.0,38.0,2.0,1.0,2.0,2.0,0.0,0.0,247.0,352.0,125.0,191.0,1.15,-113.0,1.0,1.0,0.5,6.333333,3.676471,9.263158,7.958333,5,5
1,21,12,400.0,199.0,39.0,14.0,29.0,41.0,2.0,3.0,1.0,1.0,2.0,2.0,218.0,176.0,193.0,39.0,0.74,201.0,-1.0,1.0,1.0,7.517241,4.948718,4.292683,2.785714,3,1


In [106]:
y = combined_df[['win']]

In [107]:
model = LinearRegression().fit(X, y)

In [108]:
fig = go.Figure(data = go.Bar(name = 'variable coefficients', x = dep_var, y = list(model.coef_[0])))

layout = go.Layout(
    title = 'Model Feature Coeficients',
    xaxis= dict(title= 'Dependent Variables',ticklen= 1,zeroline= False),
    yaxis= dict(title= 'Coef',ticklen= 5,zeroline= False)
)

fig.update_layout(
    title={
        'text':'Model Feature Coefficients',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig

From above, we see that some variable appear to be more important than others such as, team against, score for and score against (which may be impacted by weather), pass/run ratio, turnover differential, and pass plays against versus run plays against. Using a basic model with some of these variables and impacts from weather should help us create some kind of game wins prediction.

<h2>Variables to Explore Affects of Weather On</h2><br>
1. Score<br>
2. Score against<br>
3. pass/run ratio<br>
4. to_diff<br>
5. FG Attempts<br>
6. FG Success Rate<br>

<h2>Team Splitters</h2>
1. Win Rank<br>
2. Pass/Run Ratio Category (Pass Heavy, Middle, Run Heavy)<br>
3. Typical Bad Weather Teams versus Good Weather or Dome<br>

In [109]:
combined_df.to_csv('nfl_small_end_of_sum.csv')

<h2>More Variables Exploration to Include in Summary DF</h2>

In [ ]:
nfl_small2_end_of_drive[nfl_small2_end_of_drive['field_goal_result'].notnull()].head(3)

In [ ]:
#Can include field goals attempted and made
nfl_small2_end_of_drive.field_goal_result.unique()

In [ ]:
nfl_small2_end_of_drive[(nfl_small2_end_of_drive['game_id']==2017090700)&\
                        (nfl_small2_end_of_drive['home_team']=='NE')&\
                        (nfl_small2_end_of_drive['play_type']=='field_goal')
                       ].field_goal_result.count()

In [ ]:
nfl_small2_end_of_drive[(nfl_small2_end_of_drive['game_id']==2017090700)&\
                        (nfl_small2_end_of_drive['home_team']=='NE')
                       ].field_goal_result.count()

<h2>Dashboard Scrap</h2>

In [ ]:
game_dict = df.apply(lambda x : [x['game_id'], x['game_name']],axis=1)

In [ ]:
game_dict

In [ ]:
# [i for i in game_dict]

In [ ]:
nfl_small2_end_of_drive['cum_yds'] = nfl_small2_end_of_drive.groupby(['game_id'])['ydsnet'].cumsum()

In [ ]:
# Getting data from one game to explore further
kc_ne_game = nfl_small2[nfl_small2['game_id']==2017090700].sort_values(by='play_id',ascending=True)

In [ ]:
kc_ne_game.head(5)

In [ ]:
kc_ne_game.shape

In [ ]:
# Creating end of drive column to easily sum data

for i in range(0, len(kc_ne_game)-1):
    kc_ne_game.loc[i,'end_of_drive'] = kc_ne_game.loc[i+1,'drive'] - kc_ne_game.loc[i,'drive']

# Adding one more for last drive of game
kc_ne_game.loc[len(kc_ne_game)-1,'end_of_drive'] = 1

In [ ]:
# Ensuring end of drive column is created correctly
kc_ne_game.head(5)

In [ ]:
# Pulling end of drive rows only
kc_ne_game[kc_ne_game['end_of_drive'] == 1].head()

In [ ]:
# There appears to be something wrong with yardage data for TEN
# https://www.espn.com/nfl/playbyplay?gameId=290910023
kc_ne_game[(kc_ne_game['end_of_drive'] == 1)&(kc_ne_game['game_half'] == 'Half1')]\
                                            .groupby(['posteam','drive']).agg({'ydsnet':np.sum})

In [ ]:
kc_ne_game[(kc_ne_game['end_of_drive'] == 1)&(kc_ne_game['game_half'] == 'Half1')]\
                                            .groupby(['posteam','drive'])['ydsnet'].cumsum()

In [ ]:
trace1 = go.Scatter(
      x = kc_ne_game['drive'], 
      y = kc_ne_game['total_home_score'], 
      mode = 'lines', 
      name = 'New England',
      type = "scatter"
    )

trace2 = go.Scatter(
      x = kc_ne_game['drive'], 
      y = kc_ne_game['total_away_score'], 
      mode = "lines", 
      name = 'Kansas City',
      type = "scatter"
    )

data = [trace1,trace2]

In [ ]:
layout = go.Layout(
    title = 'End of Drive Score',
    xaxis= dict(title= 'Drive',ticklen= 1,zeroline= False),
    yaxis= dict(title= 'Points',ticklen= 5,zeroline= False)
)

fig = go.Figure(data = data, layout = layout)

fig.update_layout(
    title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig

<h2>Matplotlib Version</h2>

In [ ]:
# Reset Fig
plt.close('all')
plt.style.use('seaborn')
fig, ax1 = plt.subplots(figsize=(20,7))
# plt.gca().invert_xaxis()

line1 = ax1.plot(kc_ne_game['drive'],kc_ne_game['total_home_score'],'y--',label='PIT')
line2 = ax1.plot(kc_ne_game['drive'],kc_ne_game['total_away_score'],'b-',label='TEN')

ax1.set_xlabel('Drive Number',fontsize=14)
ax1.set_ylabel('End of Drive Score',fontsize=14)
ax1.set_title("End of Drive Score",fontsize=14,fontweight='bold')

ax1.legend(loc=4,fontsize=16)

plt.show()

<h2>Combining weather data with dataframe data</h2>

In [ ]:
import os
import re
def load_weather():

    # mapping out each stadium with their own weather dataframe in a dictionary
       
    weather_path = os.getcwd() + '\\historicalweatherdata'
    files = os.listdir(weather_path)
    df = pd.DataFrame()
    for name in files:
        if '.csv' in name and len(name) <= 7:
            
            df2 = pd.read_csv(weather_path + '\\' + name)
            df = pd.concat([df, df2], sort=True)
    
    df = df.drop(['time_local'], axis=1)
    df['Time (GMT)'] = pd.to_datetime(df['Time (GMT)'])
    df['Time (EST)'] = pd.to_datetime(df['Time (EST)'])
    
    df = df.sort_values(by=['Team Abbreviation', 'Time (EST)'])
    
    return df

In [ ]:
weather_df = load_weather()

In [ ]:
weather_df.head(10)

In [ ]:
weather_df.dtypes

In [ ]:
nfl_small2.head()

In [ ]:
nfl_small2['Estimated_Hour'] = nfl_small2['Estimated_Time'].dt.round('H')
nfl_weather = pd.merge(nfl_small2, weather_df, how = 'left', left_on = ['home_team', 'Estimated_Hour'], right_on = ['Team Abbreviation', 'Time (EST)'])
nfl_weather.sample(10)

In [ ]:
nfl_weather.dtypes

In [ ]:
nfl_weather.info()

In [ ]:
nfl_small2.info(), weather_df.info()

In [ ]:
with open ('stadium_dict.pkl', 'rb') as handle:
    stadium_dict = pickle.load(handle)

for k, v in stadium_dict.items():
    print(k,v)

In [ ]:
print(nfl_weather['Roof'].info())
nfl_weather['Roof'] = nfl_weather.apply(lambda x: stadium_dict[x['Hmoe']])


In [ ]:
nfl_weather.info()

In [ ]:
nfl_weather[nfl_weather[['Roof', 'Team Abbreviation']].isnull().any(axis=1)].sample(200)